importing required packages & Reading files from folder

In [ ]:
### importing required libraries...............
import pandas as pd
import os
import glob
import re
import numpy as np
import xlwings as xw
from xlwings import Range, constants
import time
from datetime import datetime
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

#Get the List of files from path
os.chdir(r"D:\Zigna AI Corp\Zigna AI Corp - Hospital Application_2022-03-09\21 files")

path = os.getcwd()
files = os.listdir(path)
#List of Files 
mylinks=[]
for l in files:
    l
    mylinks.append(l)
mylinks  

cols=['FILE_NAMES']
#Creating the dataframe
CNames=pd.DataFrame(mylinks,columns=cols)
CNames

#extracting ids and type of file from CNames dataframe
CNames[['ID']]=CNames.FILE_NAMES.str.extract('(\d+)')
CNames['Type'] = CNames.FILE_NAMES.str.split('.',expand=True)[1]
CNames
CNames['Errors'] = ""


Readign files and doing all preprocessing techniques

In [ ]:
combined_final1 = []
combined_data= []
dataframe = []
count = 0
raw = []
start = time.time()
for k in CNames.itertuples(index = False):
    f = k[0]
    combined_final = []
    if k[2] == 'csv':
        try:
            df =pd.read_csv(f)
            df['Hospital_Id'] = k[1]
            dataframe.append(df)
        except:
            df=pd.read_csv(f,encoding='latin1')
            df['Hospital_Id'] = k[1]
            dataframe.append(df)
            
    elif k[2] == 'xlsx':
        df = pd.read_excel(f)
        df['Hospital_Id'] = k[1]
        dataframe.append(df)
    else:
        None
    
    df1 = df.columns
    df2 = pd.Series(df1)
    raw_colnames = df2.to_frame(name="Columns")
    raw_colnames['ID'] = k[1]
    
    ################################   cleaing raw column names array          ###################################
    # raw_colnames['Columns'] = raw_colnames['Columns'].str.replace('#' ,'nmbr')
    raw_colnames['Columns'] = raw_colnames['Columns'].map(lambda x: str(x.lower().strip()))
    raw_colnames['Columns'] = raw_colnames['Columns'].str.replace('[^\w\s]',' ')
    raw_colnames['Columns'] = raw_colnames['Columns'].str.replace('_',' ')
    raw_colnames['Columns'] = raw_colnames['Columns'].str.replace(' +', ' ')        
    raw_colnames['Columns'] = raw_colnames['Columns'].str.replace('\n', ' ')        
    raw_colnames['Category'] = np.nan
    

Passing through the exceptions(hospital id & sheet name), cost column strings and exclude strings on the raw columns.

In [ ]:
    for j in exceptions_str:
        raw_colnames['Category'] = np.where((raw_colnames['Columns'] == j), raw_colnames['Columns'], raw_colnames['Category'])
    
    cost_list = pd.read_excel(r"D:\Copy of Json Renaming Local.xlsx",sheet_name='name')
    cost_list1 = cost_list['Cost_columns']
    cost_str = [item for item in cost_list1 if not(pd.isna(item)) == True]
    
    exclude_list = cost_list['Exclude']
    exclude_str = [item for item in exclude_list if not(pd.isna(item)) == True]
    
    ##########################  mapping with cost strings with string contains and exclude list   ###########################
    for x in cost_str:
        raw_colnames['Category'] = np.where((raw_colnames['Columns'].str.contains(x)), "name", raw_colnames['Category'])
    
    for e in exclude_str:
        raw_colnames['Category'] = np.where((raw_colnames['Columns'].str.contains(e)), np.nan, raw_colnames['Category'])
    
    ################################################################################################################
    ######################taking mapped columns and appending into list  #############################################
    cost_mappings = raw_colnames[raw_colnames['Category'].notnull()]
    cost_mappings['type'] = 'cost strings mapping'
    combined_final.append(cost_mappings)


doing Hard match for unmapped columns with lookup values 

In [ ]:
    #########################################   Hard Match  ##########################################################
    raw_colnames1 = raw_colnames[raw_colnames['Category'].isnull()]
    for (colname,colval) in patterns.iteritems():
        colvalue = [item for item in colval if not(pd.isna(item)) == True]
        for index,j in raw_colnames1['Columns'].iteritems():
            if pd.isnull(raw_colnames1['Category'][index]):
                try:                
                    Sheet = [ele for ele in colvalue if(ele == j)][0]
                    raw_colnames1['Category'][index] = colname
                except:
                    None
    #################################################################################################################
    Unmap_columns = raw_colnames1[raw_colnames1['Category'].isnull()]
    col_list = Unmap_columns['Columns'].values.tolist()
    Unmapped_columns1 = pd.Series(col_list)

    map_columns = raw_colnames1[raw_colnames1['Category'].notnull()]
    map_columns['type'] = 'Hard Match'
    combined_final.append(map_columns)

Taking unmapped columns from the above steps and doing patterns with three conditions like
    Mapping by adding trailing and leading underscores.
    Mapping by adding leading underscore 
    Mapping by adding trailing underscore

In [ ]:
################# cleaning unmapped columns cleaning for patterns match  ########################################
    Unmap_columns['Columns'] = Unmap_columns['Columns'].str.replace('[^\w\s]','_')
    Unmap_columns['Columns'] = Unmap_columns['Columns'].map(lambda x: '_' + str(x.lower().strip()) + '_')
    Unmap_columns['Columns'] = Unmap_columns['Columns'].str.replace(" ", "_")
    Unmap_columns['Columns'] = Unmap_columns['Columns'].str.replace('_+', '_')
    
    ######################  adding leading $ trailing underscores for pattern match  ################################
    patterns1 = patterns.copy()
    for u in patterns1.columns:
        patterns1[u] = patterns1[u].astype('str')
        patterns1[u] = patterns1[u].str.strip()
        patterns1[u] = patterns1[u].str.lower()
    
    for v in patterns1.columns:
        patterns1[v] = patterns1[v].map(lambda x: '_' + x + '_')
        patterns1[v] = patterns1[v].map(lambda x: str(x.replace(" ", "_")))
        patterns1[v] = patterns1[v].str.replace('_+', '_')
    ###############################################################################################################
    
    ################  patterns match with trailing & leading underscores  #########################################
    colnum = pd.read_excel(r"D:\Copy of Json Renaming Local.xlsx",sheet_name='column_namelist')
    for (colname,colval) in patterns1.iteritems():
        colvalue = [item for item in colval if not(pd.isna(item)) == True or item == '_nan_']
        for index,j in Unmap_columns['Columns'].iteritems():
            if pd.isnull(Unmap_columns['Category'][index]):
                try:
                    Sheet = [ele for ele in colvalue if(ele in j)][0]
                    Unmap_columns['Category'][index] = colname
                except:
                    None
                    
    ############################################################################################################
    
    #####################  patterns match with only trailing underscore  ##########################################
    # Mapping by adding leading underscore    
    patterns2 = patterns.copy()
    for u in patterns2.columns:
        patterns2[u] = patterns2[u].astype('str')
        patterns2[u] = patterns2[u].str.strip()
        patterns2[u] = patterns2[u].str.lower()
    
    for v in patterns2.columns:
        patterns2[v] = patterns2[v].map(lambda x: '_' + x)
        patterns2[v] = patterns2[v].map(lambda x: str(x.replace(" ", "_")))
        patterns2[v] = patterns2[v].str.replace('_+', '_')
    
    for (colname,colval) in patterns2.iteritems():
        colvalue = [item for item in colval if not(pd.isna(item)) == True or item == '_nan']
        for index,j in Unmap_columns['Columns'].iteritems():
            if pd.isnull(Unmap_columns['Category'][index]):
                try:
                    Sheet = [ele for ele in colvalue if(ele in j)][0]
                    Unmap_columns['Category'][index] = colname
                except:
                    None
    ##################################################################################################################

    ################  patterns match with only leading underscore  ###############################################
    #Mapping by adding trailing underscore.
    patterns3 = patterns.copy()
    for u in patterns3.columns:
        patterns3[u] = patterns3[u].astype('str')
        patterns3[u] = patterns3[u].str.strip()
        patterns3[u] = patterns3[u].str.lower()
    
    for v in patterns3.columns:
        patterns3[v] = patterns3[v].str.replace(' +', ' ')
        patterns3[v] = patterns3[v].map(lambda x: x + '_')
        patterns3[v] = patterns3[v].map(lambda x: str(x.replace(" ", "_")))
        patterns3[v] = patterns3[v].str.replace('_+', '_')
    
    for (colname,colval) in patterns3.iteritems():
        colvalue = [item for item in colval if not(pd.isna(item)) == True or item == 'nan_']
        for index,j in Unmap_columns['Columns'].iteritems():
            if pd.isnull(Unmap_columns['Category'][index]):
                try:
                    Sheet = [ele for ele in colvalue if(ele in j)][0]
                    Unmap_columns['Category'][index] = colname
                except:
                    None  
    ################################################################################################################## 
    

Modifier check: checking for the fourth character it should be(I/_/numeric) 

In [ ]:
    Unmap_columns = Unmap_columns.reset_index(drop=True)
    ############################## modifier exception   #########################################################
    for index, x in Unmap_columns['Category'].iteritems():
        if x == 'modifier':
            raw_name = Unmap_columns['Columns'][index]
            
            s = (raw_name.split("mod",1)[1])[0]
            
            if s == '_' or s == 'i' or s.isdigit():
                Unmap_columns['Category'][index] = 'modifier'
            else:
                Unmap_columns['Category'][index] = 'name'

Data type check for (inpatient-outpatient, payer name, benefit type): By raw column datatype and top 100 values check

In [ ]:
    df.columns = df.columns.str.replace('[^\w\s]','_')
    df.columns = df.columns.map(lambda x: '_' + str(x.lower().strip()) + '_')
    df.columns = df.columns.str.replace('\n', ' ')        
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.replace('_+', '_')
    ########################taking sub dataframe with 500 rows    ##############################################################
    
    # if df.shape[0] > 500:
    #     df1 = df.iloc[:500,:]
    # else:
        # df1 = df

    #################  type check list (ip-op, payername, benefit type)   ########################################
    
    Dtype_list = pd.read_excel(r"D:\Copy of Json Renaming Local.xlsx",sheet_name='Dtype check list')
    Dtype_str = Dtype_list['Category']   
    
    ###########################  type check code for above 3 categories   #####################################################
    for x in Dtype_str:
        for index, j in Unmap_columns['Category'].iteritems():
            if j == x:
                column = df[Unmap_columns['Columns'][index]].to_frame()
                column = column.dropna(axis = 0,inplace = False)
                column = column.drop_duplicates()
                   
                column1 = column.head(500)
                column1 = column1.astype(str)
                try:
                    column1[Unmap_columns['Columns'][index]] = column1[Unmap_columns['Columns'][index]].str.replace('*', ' ')
                    column1[Unmap_columns['Columns'][index]] = column1[Unmap_columns['Columns'][index]].str.replace('$', ' ')
                except:
                    column1[Unmap_columns['Columns'][index]] = column1[Unmap_columns['Columns'][index]].str.replace('$', ' ')
                    
                else:
                    column1[Unmap_columns['Columns'][index]] = column1[Unmap_columns['Columns'][index]].str.replace('*', ' ')
                    
                try:
                    column1[Unmap_columns['Columns'][index]] = column1[Unmap_columns['Columns'][index]].str.strip()
                except:
                    None
                    
                for index1, a in column1[Unmap_columns['Columns'][index]].iteritems():
                    try:
                        a1 = float(a)
                        column1[Unmap_columns['Columns'][index]][index1] = a1
                    except:
                        None
                        
                if df[Unmap_columns['Columns'][index]].dtypes == object and len([ele for ele in column1[Unmap_columns['Columns'][index]] if pd.isna(ele) == False and type(ele) != str]) == 0:
                    Unmap_columns['Category'][index] = Unmap_columns['Category'][index]
                elif df[Unmap_columns['Columns'][index]].dtypes == object and len([ele for ele in column1[Unmap_columns['Columns'][index]] if pd.isna(ele) == False and type(ele) != str]) > 0:
                    Unmap_columns['Category'][index] = 'name'
                else:
                    Unmap_columns['Category'][index] = 'name'
            else:
                None
                
    patt_map_columns = Unmap_columns[Unmap_columns['Category'].notnull()]
    patt_map_columns['type'] = 'patterns match'
    combined_final.append(patt_map_columns)
    
    patt_unmap_columns = Unmap_columns[Unmap_columns['Category'].isnull()]
    col_list = patt_unmap_columns['Columns'].values.tolist()
    patt_unmap_columns['Category'] = np.nan

By taking remaining unmapped columns from the above steps we are doing fuzzy match algorithm

fuzzy match algorithm: fuzz token sort ratio with 80% threshold.

In [ ]:
  ####################################### fuzzy match   ############################################################
    myList1 = patt_unmap_columns['Columns'].tolist()
    
    for i in patterns.columns:
        myList2 = [item for item in patterns[i] if not(item == '_nan_' or pd.isna(item) == True)]
        threshold = 80
        match1 = []
        
        for x in myList1:
            match1.append(process.extractOne(x, myList2, scorer=fuzz.token_sort_ratio))
        
        patt_unmap_columns['matches'] = match1
        
        for index, j in patt_unmap_columns['matches'].iteritems():
            if j[1] >= threshold:
                if pd.isnull(patt_unmap_columns['Category'][index]):
                    patt_unmap_columns['Category'][index] = i
    
    fuzz_map_columns = patt_unmap_columns[patt_unmap_columns['Category'].notnull()]
    fuzz_map_columns['type'] = 'fuzz match'
    combined_final.append(fuzz_map_columns)

Making all unmapped columns into name category

In [ ]:
    ###########  Final unmapped columns into name category  ##########################################################
    fuzz_unmap_columns = patt_unmap_columns[patt_unmap_columns['Category'].isnull()]
    fuzz_unmap_columns = fuzz_unmap_columns.drop(['matches'], axis=1)
    fuzz_unmap_columns['Category'] = np.where(fuzz_unmap_columns['Category'].isnull() , 'name', fuzz_unmap_columns['Category'])
    fuzz_unmap_columns['type'] = 'Final name'
    ####################################################################################################################    
    

Various checks in final name category.

inpatient outpatient category check in final name category by passing inpatient outpatient lookup values.

In [ ]:
  #########  mapping with inpatient outpatient lookup values in final name category  ###############################
    ipop_list = pd.read_excel(r"D:\Copy of Json Renaming Local.xlsx",sheet_name='inpatient-outpatient')
    ipop_str = ipop_list['ipop_columns']   
    ipop_str = ipop_str.str.replace('_' , ' ')
    ipop_str = ipop_str.str.strip()
    
    # df1 = df.copy()
    # df1 = df1.dropna(axis="columns", how="any")
    
    fuzz_unmap_columns = fuzz_unmap_columns.reset_index(drop= True)
    for index,j in fuzz_unmap_columns['Columns'].iteritems():
        print(df[j])
        column = df[j].to_frame()
        column = column.dropna(axis = 0,inplace = False)
        column = column.drop_duplicates()
        for v in column.columns:
            column[v] = column[v].astype(str)
            column[v] = column[v].str.lower()
            
        column1 = column.head(20)
        column1 = column1.astype(str)
        for k in ipop_str:
            for i in column1[j]:
                # print(i)
                if str(i)==str(k):
                    print(True)
                    print(column1[j])
                    fuzz_unmap_columns['Category'][index] = 'inpatient_outpatient'
            else:
                None
                
    #################################################################################################################
    

Import all reqrired filles (reference tables and stop word ref files)

In [ ]:
   
    #############################  importing all required reference files  ################################################
    df1 = pd.read_excel(r"D:\Zigna AI Corp\Zigna AI Corp - Hospital Application_2022-03-09\Structured files\Code library\Codes\Consumer Friendly Descriptors.xlsx")
    df1['CPT Code'] = df1['CPT Code'].apply(lambda x: str(x).zfill(5))
    
    df2=pd.read_excel(r"D:\Zigna AI Corp\Zigna AI Corp - Hospital Application_2022-03-09\Structured files\Code library\Codes\HCPC codes till 9.27.2021.xlsx")
    df3=pd.read_excel(r"D:\Zigna AI Corp\Zigna AI Corp - Hospital Application_2022-03-09\Structured files\Code library\Codes\MS DRG 2008-2022.xlsx", dtype = str)
    df3['CODE'] = df3['CODE'].apply(lambda x: str(x).zfill(3))
 
    df4=pd.read_excel(r"D:\Zigna AI Corp\Zigna AI Corp - Hospital Application_2022-03-09\Structured files\Code library\Codes\APR DRG-2018-07-01_final_weights.xlsx", skiprows = 2)
    df4['APR-DRG'] = df4['APR-DRG'].apply(lambda x: str(x).zfill(4))
    df5=pd.read_excel(r"D:\Zigna AI Corp\Zigna AI Corp - Hospital Application_2022-03-09\Structured files\Code library\Codes\PCS master excel upto 2022.xlsx")
    
    ####################################################################################################################
    stopword_ref = r"D:\Zigna AI Corp\Zigna AI Corp - Hospital Application_2022-03-09\Structured files\Code library\Codes\stopword_ref_file.xlsx"
    sw_ref = pd.read_excel(stopword_ref)# This dataframe has all the values that to be stripped out from the code columns


In final name category, if category is name then we are going check with reference tables by taking top 100 raw column values and if the 50 % of the values matches with the any one of the reference tables then those columns will assigned to procedure codes.
Hierarchy of the check:
    By Taking the top 100rows (excluding nan's and dropping duplicates) from the raw column  

    We are mapping these 100 values with reference tables 

    Process we followed: 

    First, we have created empty columns like cpt,hcpcs,drg,aprdrg,icd for storing mapped codes from the refernce tables. 

    Fixed the length of refernce tables like for cpt (length 5),drg(length 3),aprdrg(length 4). 

    And mapped the raw columns with the refernce tables. 

    If any of the above 5 columns are filled with mapped values, then it will be assigned to 'procedure code' category. 

    out of 100 rows if any of the 50 rows matches with reference tables then only it will be assigned as procedure code. 

    After the above check also, we added some conditions in file level, those are briefed below. 

    Case1: 

            Here we are considering other than the final name category (i.e., Hard match, patterns match and fuzzy match) if it has procedure code assigned to any of the columns, then we will check in final name category. 

            for procedure code assigned columns, if the procedure codes are assigned to more than 1 column then, we will assign it as cost column or else it will remain same. 

    Case 2: 

            if Hard match, patterns match and fuzzy match does not have any procedure code assigned category then we will check in final name assigned procedure code category, if the procedure codes in final name is equal to 1, then we will assign it as procedure code, or else all the columns in final name category will fall into name category. 

In [ ]:
   #####################################  mappping with reference tables  ##############################################
    for index,j in fuzz_unmap_columns['Category'].iteritems():
        if str(j) == 'name':
            # print(True)
            column = df[fuzz_unmap_columns['Columns'][index]].to_frame()
            column = column.dropna(axis = 0,inplace = False)
            column = column.drop_duplicates()
            column2 = column.head(100)
            column2 = column2.astype(str)
            # column2 = column2.len(5)
            column2=column2.replace('\.0', '', regex=True)
            
            #Passing the list of unnecessary  strings to remove from the codes like CPT,MS,APR etc......
            p = re.compile('|'.join(map(re.escape, list(sw_ref["strings"].astype(str))))) # escape to handle metachars
            for i in column2.columns:
                column2[i] = column2[i].astype(str)                 # Making the values into strings in all the code columns
                column2[i] = column2[i].str.encode('ascii', 'ignore').str.decode('ascii')  # If any ascii characters are present they are encoded and then decoded immediately. So the special characters are cleaned up
                column2[i] = [p.sub('', s) for s in column2[i]]     # substitutes the matching strings from p with ''
                column2[i] = column2[i].astype(str)                 # Making the values into strings in all the code columns
                column2[i] = column2[i].str.strip()                 # strips out any spaces in the values
                # Data3[i] = Data3[i].replace('\.0', '', regex=True) # remove .0 from codes if exists
                column2[i] = column2[i].map(lambda x: re.sub(r'\W+', '', x)) # Removing special characters if we have any in the code columns

            
            column2[fuzz_unmap_columns['Columns'][index]] = column2[fuzz_unmap_columns['Columns'][index]].apply(lambda x: x[:5])
            
            # column1 = column1.str.strip()
            for v in column2.columns:
                column2[v] = column2[v].str.strip()
                
            column2[['cpt', 'hcpcs', 'drg', 'apr', 'icd']] = np.nan
            
            #getting all mapped codes
            FINAL = pd.merge(column2,df1['CPT Code'],how='left',left_on=fuzz_unmap_columns['Columns'][index],right_on='CPT Code')
            FINAL['cpt'] = np.where(FINAL['cpt'].isnull(), FINAL['CPT Code'], FINAL['cpt'])
            FINAL.drop('CPT Code', axis= 1, inplace=True)
            FINAL1 = FINAL.copy()
            
            FINAL= pd.merge(column2,df2['HCPC'],how='left',left_on=fuzz_unmap_columns['Columns'][index],right_on='HCPC')
            FINAL1['hcpcs'] = np.where(FINAL1['hcpcs'].isnull(), FINAL['HCPC'], FINAL1['hcpcs'])
            FINAL.drop('HCPC', axis= 1, inplace=True)

            df3['CODE'] = df3['CODE'].astype(str)
            FINAL = pd.merge(column2,df3['CODE'],how='left',left_on=fuzz_unmap_columns['Columns'][index],right_on='CODE')
            FINAL1['drg'] = np.where(FINAL1['drg'].isnull(), FINAL['CODE'], FINAL1['drg'])
            FINAL.drop('CODE', axis= 1, inplace=True)
            
            df4['APR-DRG'] = df4['APR-DRG'].astype(str)
            FINAL = pd.merge(column2,df4['APR-DRG'],how='left',left_on=fuzz_unmap_columns['Columns'][index],right_on='APR-DRG')
            FINAL1['apr'] = np.where(FINAL1['apr'].isnull(), FINAL['APR-DRG'], FINAL1['apr'])
            FINAL.drop('APR-DRG', axis= 1, inplace=True)
            
            FINAL = pd.merge(column2,df5['ICD 10 PCS'],how='left',left_on=fuzz_unmap_columns['Columns'][index],right_on='ICD 10 PCS')
            FINAL1['icd'] = np.where(FINAL1['icd'].isnull(), FINAL['ICD 10 PCS'], FINAL1['icd'])
            FINAL.drop('ICD 10 PCS', axis= 1, inplace=True)
            
            FINAL1["count"] = FINAL1.iloc[:,-5:].count(axis=1)
            
            # s = [ele for ele in FINAL1['count'] if ele >0] 
            if len([ele for ele in FINAL1['count'] if ele >0]) >50:
                fuzz_unmap_columns['Category'][index] = 'procedure code'
                
    ################################################################################################################
    combined_final.append(fuzz_unmap_columns)
    uni_id = pd.concat(combined_final)
    # a = a.drop_duplicates()
    uni_id = uni_id.reset_index(drop=True)
    uni_id.drop('matches', axis = 1, inplace = True)

    ############# logic for procedure code exceptions in final name category ###############################
    test = []
    test1 = []

    for k in uni_id.itertuples(index = False):
        # print(k)
        if k[3] != 'Final name':
            if k[2] == 'procedure code':
                test.append(k[2])
                
    for k in uni_id.itertuples(index = False):
        # print(k)
        if k[3] == 'Final name':
            if k[2] == 'procedure code':
                test1.append(k[2])
                
    if len(test) > 0 and len(test1) >1:
        print(len(test))
        for index,k in uni_id['type'].iteritems():
            if k == 'Final name' and uni_id['Category'][index] == 'procedure code':
                uni_id['Category'][index] = 'name'      #          k[2] = 'name'
            else:
                None
                
    elif len(test) == 0 and len(test1) > 2:
        for index,k in uni_id['type'].iteritems():
            if k == 'Final name' and uni_id['Category'][index] == 'procedure code':
                uni_id['Category'][index] = 'name'      #          k[2] = 'name'
            else:
                None
    else:
        None
    #####################################################################################################################################


Finally we are updating the lookup tables with columns mapped in (Patterns match & Fuzzy match & corrected categories in final name category.
For that we are using column_namelist sheet and appending through loop at a time.

In [ ]:
   ####### mapped column in patterns match Updation into lookup tables ########################################
    
    colcount = 0
    for i in patterns.columns:
        for r, index1 in colnum['Number'].iteritems():
            if r == colcount:
                ass = colnum['col name'][index1]
                print(ass)
        arr = []
        for index, g in uni_id['Category'].iteritems():
            # print(g)
            if i == g:
                print(True)
                arr.append(uni_id['Columns'][index])
            else:
                None
        if len(arr) > 0:
            Mapped_names= pd.Series(arr)
            app = xw.App(visible=False)
            wb = xw.Book(r"C:\Users\User\Downloads\Copy of Json Renaming Local.xlsx")
            a = wb.sheets[0].range(str(ass) + str(wb.sheets[0].cells.last_cell.row)).end('up').row
            sheet = wb.sheets["cleaned"]                                  
            sheet.range(str(ass) +'' + str(a+1)).options(index=False).value = Mapped_names
            wb.save()
            wb.close()
            app.quit()
        else:
            None
        colcount += 1

Need to check the final name category and if we have incorrect mappings we need to do manual mappings

Writing into an csv file for manual check into the same raw file folder.

In [ ]:
    #######################################################################################################################
    combined_final1.append(uni_id)
    end = time.time()

Final_Cate = pd.concat(combined_final1)

Final_Cate.to_csv(r'Category assignmentoutput.csv', index = False)